In [1]:
import os
import duckdb

from pathlib import Path

%load_ext sql

from dotenv import load_dotenv

load_dotenv()

DB_PATH = Path(os.getenv("SILVER_DUCKDB_PATH", ""))
DB_PATH.parent.mkdir(parents=True, exist_ok=True)

TMP_DIR = "/tmp/duckdb_tmp"  # make sure it exists

conn = duckdb.connect(DB_PATH)

conn.execute(f"SET temp_directory='{TMP_DIR}'")
conn.execute("SET memory_limit='50GB'")
conn.execute("SET threads=1")
conn.execute("SET preserve_insertion_order=false")

%sql conn --alias duck

Tip: You may define configurations in /home/c-cc2016/ra173608/tramita/pyproject.toml or /home/c-cc2016/ra173608/.jupysql/config.

Did not find user configurations in /home/c-cc2016/ra173608/tramita/pyproject.toml.

# Normalização de PKs

In [2]:
%%sql
ALTER TABLE bloco_senado
    ALTER COLUMN codigo_bloco TYPE BIGINT;

ALTER TABLE bloco_senado
    ADD CONSTRAINT pk_bloco PRIMARY KEY (codigo_bloco);

Running query in 'duck'

Success


In [3]:
%%sql
CREATE TABLE parlamentares_senado_dedup AS
SELECT *
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY codigo_parlamentar ORDER BY year_snapshot DESC) AS rn
    FROM parlamentar_senado
)
WHERE rn = 1;

DROP TABLE parlamentar_senado;
ALTER TABLE parlamentares_senado_dedup RENAME TO parlamentar_senado;


Running query in 'duck'

Success


In [4]:
%%sql
ALTER TABLE parlamentar_senado
    ALTER COLUMN codigo_parlamentar TYPE BIGINT;

ALTER TABLE parlamentar_senado
    ADD CONSTRAINT pk_parlamentar PRIMARY KEY (codigo_parlamentar);

Running query in 'duck'

Success


In [5]:
%%sql
ALTER TABLE partido_senado
    ALTER COLUMN codigo_partido TYPE BIGINT;

ALTER TABLE partido_senado
    ADD CONSTRAINT pk_partido PRIMARY KEY (codigo_partido);

Running query in 'duck'

Success


In [6]:
%%sql
ALTER TABLE processo_senado
    ALTER COLUMN id_processo TYPE BIGINT;

ALTER TABLE processo_senado
    ADD CONSTRAINT pk_processo PRIMARY KEY (id_processo);

Running query in 'duck'

Success


In [7]:
%%sql
ALTER TABLE emendas_senado
    ALTER COLUMN id_emenda TYPE BIGINT;

ALTER TABLE emendas_senado
    ADD CONSTRAINT pk_emenda PRIMARY KEY (id_emenda);

Running query in 'duck'

Success


In [8]:
%%sql
ALTER TABLE relatorias_senado
    ALTER COLUMN id_relatoria TYPE BIGINT;

ALTER TABLE relatorias_senado
    ADD CONSTRAINT pk_relatoria PRIMARY KEY (id_relatoria);

Running query in 'duck'

Success


In [9]:
%%sql
ALTER TABLE votacoes_senado
    ALTER COLUMN id_votacao TYPE BIGINT;

ALTER TABLE votacoes_senado
    ADD CONSTRAINT pk_votacao PRIMARY KEY (id_votacao);

Running query in 'duck'

Success


In [10]:
%%sql
ALTER TABLE votos_senado
    ALTER COLUMN id_voto TYPE BIGINT;

ALTER TABLE votos_senado
    ADD CONSTRAINT pk_voto PRIMARY KEY (id_voto);

Running query in 'duck'

Success


In [11]:
%%sql
ALTER TABLE autoria_iniciativa_senado
    ALTER COLUMN id_autoria_iniciativa TYPE BIGINT;

ALTER TABLE autoria_iniciativa_senado
    ADD CONSTRAINT pk_autoria_iniciativa PRIMARY KEY (id_autoria_iniciativa);

Running query in 'duck'

Success


In [12]:
%%sql
ALTER TABLE situacoes_senado
    ALTER COLUMN id_situacao TYPE BIGINT;

ALTER TABLE situacoes_senado
    ADD CONSTRAINT pk_situacao PRIMARY KEY (id_situacao);

Running query in 'duck'

Success


In [13]:
%%sql
SELECT t.*
FROM despachos_senado t
JOIN (
    SELECT id_despacho
    FROM despachos_senado
    GROUP BY id_despacho
    HAVING COUNT(*) > 1
) dups
ON t.id_despacho = dups.id_despacho
ORDER BY t.id_despacho;

Running query in 'duck'

id_processo,id_despacho,data_despacho,cancelado,tipo_motivacao,sigla_tipo_motivacao,year_snapshot
7412455,7824068,2018-08-09,Não,Leitura da Matéria,LEITURA_DA_MATERIA,2018
7621491,7824068,2018-08-09,Não,Leitura da Matéria,LEITURA_DA_MATERIA,2018
7693160,7880428,2018-10-30,Não,Leitura da Matéria,LEITURA_DA_MATERIA,2018
7693167,7880428,2018-10-30,Não,Leitura da Matéria,LEITURA_DA_MATERIA,2018
7693182,7880428,2018-10-30,Não,Leitura da Matéria,LEITURA_DA_MATERIA,2018
7693177,7880428,2018-10-30,Não,Leitura da Matéria,LEITURA_DA_MATERIA,2018
7693217,7880428,2018-10-30,Não,Leitura da Matéria,LEITURA_DA_MATERIA,2018
7693226,7880428,2018-10-30,Não,Leitura da Matéria,LEITURA_DA_MATERIA,2018
7703389,7893781,2018-12-10,Não,Leitura da Matéria,LEITURA_DA_MATERIA,2018
7703387,7893781,2018-12-10,Não,Leitura da Matéria,LEITURA_DA_MATERIA,2018


In [14]:
%%sql
ALTER TABLE despachos_senado
    ALTER COLUMN id_despacho TYPE BIGINT;

ALTER TABLE despachos_senado
    ALTER COLUMN id_processo TYPE BIGINT;

ALTER TABLE despachos_senado
    ADD CONSTRAINT pk_despacho PRIMARY KEY (id_despacho, id_processo);

Running query in 'duck'

Success


In [15]:
%%sql
ALTER TABLE outros_numeros_senado
    ALTER COLUMN id_outro_numero TYPE BIGINT;

ALTER TABLE outros_numeros_senado
    ADD CONSTRAINT pk_outro_numero PRIMARY KEY (id_outro_numero);

Running query in 'duck'

Success


In [16]:
%%sql
ALTER TABLE documento_autoria_senado
    ALTER COLUMN id_documento_autoria TYPE BIGINT;

ALTER TABLE documento_autoria_senado
    ADD CONSTRAINT pk_documento_autoria PRIMARY KEY (id_documento_autoria);

Running query in 'duck'

Success


In [17]:
%%sql
ALTER TABLE processos_relacionados_senado
    ALTER COLUMN id_processo_relacionado TYPE BIGINT;

ALTER TABLE processos_relacionados_senado
    ADD CONSTRAINT pk_processo_relacionado PRIMARY KEY (id_processo_relacionado);

Running query in 'duck'

Success


In [18]:
%%sql
ALTER TABLE autuacoes_senado
    ALTER COLUMN autuacao_idx TYPE BIGINT;

ALTER TABLE autuacoes_senado
    ADD CONSTRAINT pk_autuacao PRIMARY KEY (id_processo, autuacao_idx);

Running query in 'duck'

Success


In [19]:
%%sql
ALTER TABLE informes_legislativos_senado
    ALTER COLUMN id_informe_legislativo TYPE BIGINT;

ALTER TABLE informes_legislativos_senado
    ADD CONSTRAINT pk_informe_legislativo PRIMARY KEY (id_informe_legislativo);

Running query in 'duck'

Success


In [20]:
%%sql
ALTER TABLE informes_documentos_associados_senado
    ALTER COLUMN id_documento_associado TYPE BIGINT;

ALTER TABLE informes_documentos_associados_senado
    ADD CONSTRAINT pk_documento_associado PRIMARY KEY (id_documento_associado);

Running query in 'duck'

Success


In [21]:
%%sql
ALTER TABLE providencias_senado
    ALTER COLUMN id_providencia TYPE BIGINT;

ALTER TABLE providencias_senado
    ADD CONSTRAINT pk_providencia PRIMARY KEY (
        id_processo,
        id_despacho,
        id_providencia
    );

Running query in 'duck'

Success


In [22]:
%%sql
ALTER TABLE movimentacoes_senado
    ADD CONSTRAINT pk_movimentacao PRIMARY KEY (
        id_processo,
        autuacao_idx,
        movimentacao_idx
    );

Running query in 'duck'

Success


In [23]:
%%sql
ALTER TABLE unidades_destinatarias_senado
    ALTER COLUMN id_unidade_destinataria TYPE BIGINT;

ALTER TABLE unidades_destinatarias_senado
    ADD CONSTRAINT pk_unidade_destinataria PRIMARY KEY (id_unidade_destinataria);

Running query in 'duck'

Success


In [24]:
%%sql
SELECT t.*
FROM encontro_legislativo_senado t
JOIN (
    SELECT id_encontro_legislativo
    FROM encontro_legislativo_senado
    GROUP BY id_encontro_legislativo
    HAVING COUNT(*) > 1
) dups
ON t.id_encontro_legislativo = dups.id_encontro_legislativo
ORDER BY t.id_encontro_legislativo;

Running query in 'duck'

id_processo,id_despacho,id_encontro_legislativo,data_encontro,tipo_encontro,descricao_encontro,casa_encontro,numero_encontro,colegiado_casa,colegiado_codigo,colegiado_nome,colegiado_sigla,year_snapshot
7620082,7821741,7619828,2018-08-07,SES,Sessão Deliberativa Ordinária,SF,110,SF,1998,Plenário do Senado Federal,PLEN,2018
7620084,7821831,7619828,2018-08-07,SES,Sessão Deliberativa Ordinária,SF,110,SF,1998,Plenário do Senado Federal,PLEN,2018
7620086,7821866,7619828,2018-08-07,SES,Sessão Deliberativa Ordinária,SF,110,SF,1998,Plenário do Senado Federal,PLEN,2018
7620120,7821808,7619828,2018-08-07,SES,Sessão Deliberativa Ordinária,SF,110,SF,1998,Plenário do Senado Federal,PLEN,2018
7620163,7821850,7619828,2018-08-07,SES,Sessão Deliberativa Ordinária,SF,110,SF,1998,Plenário do Senado Federal,PLEN,2018
7620343,7822123,7619828,2018-08-07,SES,Sessão Deliberativa Ordinária,SF,110,SF,1998,Plenário do Senado Federal,PLEN,2018
7412505,7821457,7619828,2018-08-07,SES,Sessão Deliberativa Ordinária,SF,110,SF,1998,Plenário do Senado Federal,PLEN,2018
7621240,7823122,7620806,2018-08-08,SES,Sessão Deliberativa Ordinária,SF,111,SF,1998,Plenário do Senado Federal,PLEN,2018
7412485,7822751,7620806,2018-08-08,SES,Sessão Deliberativa Ordinária,SF,111,SF,1998,Plenário do Senado Federal,PLEN,2018
7621131,7823020,7620806,2018-08-08,SES,Sessão Deliberativa Ordinária,SF,111,SF,1998,Plenário do Senado Federal,PLEN,2018


In [25]:
%%sql
CREATE TABLE encontro_legislativo_senado_dedup AS
SELECT *
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY id_encontro_legislativo ORDER BY year_snapshot DESC) AS rn
    FROM encontro_legislativo_senado
)
WHERE rn = 1;

DROP TABLE encontro_legislativo_senado;
ALTER TABLE encontro_legislativo_senado_dedup RENAME TO encontro_legislativo_senado;


Running query in 'duck'

Success


In [26]:
%%sql
ALTER TABLE encontro_legislativo_senado
    ALTER COLUMN id_encontro_legislativo TYPE BIGINT;

ALTER TABLE encontro_legislativo_senado
    ADD CONSTRAINT pk_encontro_legislativo PRIMARY KEY (
        id_encontro_legislativo
    );

Running query in 'duck'

Success


# Deduplicação de tabelas relacionais

In [27]:
%%sql
SELECT a.*
FROM informes_legislativos_senado a
JOIN (
    SELECT id_processo, id_informe
    FROM informes_legislativos_senado
    GROUP BY id_processo, id_informe
    HAVING COUNT(*) > 1
) dups
ON  a.id_processo = dups.id_processo
-- AND a.id_ente = dups.id_ente
-- AND a.autor = dups.autor
-- AND a.sigla_ente = dups.sigla_ente
AND a.id_informe = dups.id_informe
ORDER BY a.id_processo, a.id_informe


Running query in 'duck'

id_informe_legislativo,id_processo,id_informe,data_informe,descricao,id_situacao_iniciada,sigla_situacao_iniciada,ente_adm_casa,ente_adm_id,ente_adm_nome,ente_adm_sigla,colegiado_casa,colegiado_codigo,colegiado_nome,colegiado_sigla,year_snapshot
41209,500398,2119354,2023-03-08,"Aguardando inclusão em Ordem do Dia do Requerimento nº 127, de 2023, do Senador Veneziano Vital do Rêgo e outros, solicitando o desarquivamento da matéria.",95,AGINCL(RQ),SF,55312,Secretaria Legislativa do Senado Federal,SLSF,SF,1998,Plenário do Senado Federal,PLEN,2017
41871,500398,2119354,2023-03-08,"Aguardando inclusão em Ordem do Dia do Requerimento nº 127, de 2023, do Senador Veneziano Vital do Rêgo e outros, solicitando o desarquivamento da matéria.",95,AGINCL(RQ),SF,55312,Secretaria Legislativa do Senado Federal,SLSF,SF,1998,Plenário do Senado Federal,PLEN,2017
42504,500398,2119354,2023-03-08,"Aguardando inclusão em Ordem do Dia do Requerimento nº 127, de 2023, do Senador Veneziano Vital do Rêgo e outros, solicitando o desarquivamento da matéria.",95,AGINCL(RQ),SF,55312,Secretaria Legislativa do Senado Federal,SLSF,SF,1998,Plenário do Senado Federal,PLEN,2017
12940,502077,2080636,2022-03-16,"Concluída a instrução das matérias, fica aberto o prazo de cinco dias úteis para recebimento de emendas, nos termos do art. 235, II, ""d"", do Regimento Interno.",88,AGREMESA,SF,13594,Secretaria de Atas e Diários,SEADI,SF,1998,Plenário do Senado Federal,PLEN,2016
10909,502077,2080636,2022-03-16,"Concluída a instrução das matérias, fica aberto o prazo de cinco dias úteis para recebimento de emendas, nos termos do art. 235, II, ""d"", do Regimento Interno.",88,AGREMESA,SF,13594,Secretaria de Atas e Diários,SEADI,SF,1998,Plenário do Senado Federal,PLEN,2016
20249,504573,2119092,2023-03-03,"Aguardando inclusão em Ordem do Dia do Requerimento nº 102, de 2023, da Senadora Professora Dorinha Seabra e outros, solicitando o desarquivamento da matéria.",95,AGINCL(RQ),SF,55312,Secretaria Legislativa do Senado Federal,SLSF,SF,1998,Plenário do Senado Federal,PLEN,2015
22020,504573,2119092,2023-03-03,"Aguardando inclusão em Ordem do Dia do Requerimento nº 102, de 2023, da Senadora Professora Dorinha Seabra e outros, solicitando o desarquivamento da matéria.",95,AGINCL(RQ),SF,55312,Secretaria Legislativa do Senado Federal,SLSF,SF,1998,Plenário do Senado Federal,PLEN,2015
23716,504573,2119092,2023-03-03,"Aguardando inclusão em Ordem do Dia do Requerimento nº 102, de 2023, da Senadora Professora Dorinha Seabra e outros, solicitando o desarquivamento da matéria.",95,AGINCL(RQ),SF,55312,Secretaria Legislativa do Senado Federal,SLSF,SF,1998,Plenário do Senado Federal,PLEN,2015
25308,504573,2119092,2023-03-03,"Aguardando inclusão em Ordem do Dia do Requerimento nº 102, de 2023, da Senadora Professora Dorinha Seabra e outros, solicitando o desarquivamento da matéria.",95,AGINCL(RQ),SF,55312,Secretaria Legislativa do Senado Federal,SLSF,SF,1998,Plenário do Senado Federal,PLEN,2015
26789,504573,2119092,2023-03-03,"Aguardando inclusão em Ordem do Dia do Requerimento nº 102, de 2023, da Senadora Professora Dorinha Seabra e outros, solicitando o desarquivamento da matéria.",95,AGINCL(RQ),SF,55312,Secretaria Legislativa do Senado Federal,SLSF,SF,1998,Plenário do Senado Federal,PLEN,2015


In [28]:
%%sql
SELECT SUM(c) AS total_duplicadas
FROM (
    SELECT COUNT(*) - 1 AS c
    FROM informes_legislativos_senado
    GROUP BY id_processo, id_informe, descricao, data_informe, sigla_situacao_iniciada, data_informe
    HAVING COUNT(*) > 1
) t;

Running query in 'duck'

total_duplicadas
158784


In [29]:
%%sql
DELETE FROM informes_legislativos_senado
WHERE rowid IN (
    SELECT rowid
    FROM (
        SELECT 
            rowid,
            ROW_NUMBER() OVER (
                PARTITION BY id_processo, id_informe, descricao, data_informe, sigla_situacao_iniciada, data_informe
                ORDER BY rowid
            ) AS rn
        FROM informes_legislativos_senado
    ) t
    WHERE rn > 1
);

Running query in 'duck'

Count
158784
